In [1]:
import pandas as pd
import scanpy as sc
import os
import numpy as np

In [2]:
datadir = '/home/jing/Phd_project/project_UCD_blca/blca_publication_OUTPUT/'
os.chdir(datadir)

In [3]:
mtx =os.path.join('blca_publication_OUTPUT_sct/',"sct_corrected_UMI.mtx")
cells=pd.read_csv(os.path.join('blca_publication_OUTPUT_sct/',"sct_corrected_UMI_cells.txt"),header=None,index_col=0)
features=pd.read_csv(os.path.join('blca_publication_OUTPUT_sct/','sct_corrected_UMI_genes.txt'),header=None,sep='\t',index_col=0)
adata = sc.read_mtx(mtx)


In [4]:
adata.obs['CellID']= cells.index.tolist()
adata.var['Gene']= features.index.tolist()
adata.var.index= adata.var['Gene']
display(adata)
print(f'Max val of sct matrix before transformation, {np.max(adata.X)}')

AnnData object with n_obs × n_vars = 15954 × 19647
    obs: 'CellID'
    var: 'Gene'

Max val of sct matrix before transformation, 8.334712028503418


### Merging converted human genes

In [5]:
m_h_conversion = pd.read_csv('/home/jing/Phd_project/project_UCD_blca/blca_OUTPUT/blca_OUTPUT_m_h_convert/m_h_convertion.csv',index_col='Mouse_Genes')
display(m_h_conversion)

,Unnamed: 0,Human_Genes
Mouse_Genes,,
Aars,1,AARS
Abcb6,2,ABCB6
Abcc5,3,ABCC5
Abcf1,4,ABCF1
Abcf3,5,ABCF3
...,...,...
Trp53,961,TP53
Trp53bp1,962,TP53BP1
Zfp131,963,ZNF131


In [6]:
common_mouse_genes= list(set(adata.var.index).intersection(set(m_h_conversion.index)))
len(common_mouse_genes)

952

In [7]:
# Iterate over common genes and use .at for proper assignment
adata.var['Lincs'] = np.nan
for i in common_mouse_genes:
    if i in m_h_conversion.index:
        # Use .at for efficient and correct value assignment
        adata.var.at[i, 'Lincs'] = m_h_conversion.at[i, 'Human_Genes']

display(adata.var)
len(adata.var[adata.var['Lincs'].notnull()])

/tmp/ipykernel_346286/2142008116.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'INPP4B' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  adata.var.at[i, 'Lincs'] = m_h_conversion.at[i, 'Human_Genes']


,Gene,Lincs
Gene,,
Xkr4,Xkr4,NaN
Sox17,Sox17,NaN
Mrpl15,Mrpl15,NaN
Lypla1,Lypla1,LYPLA1
Gm37988,Gm37988,NaN
...,...,...
Gm47936,Gm47936,NaN
Gm29595,Gm29595,NaN
Fgf8,Fgf8,NaN


952

In [8]:
gene_info_df=pd.read_csv('/home/jing/Phd_project/project_GBM/gbm_DATA/gbm_DATA_LINCS/2020/info/geneinfo_beta.txt', sep = "\t", index_col = 1)

gene_info_df = gene_info_df.sort_values(by = ["gene_symbol"])
display(gene_info_df)

lincs_set = set(str(x) for x in adata.var['Lincs'].tolist())
landmark_set = set(str(x) for x in gene_info_df[gene_info_df['feature_space'] == 'landmark'].index.tolist())
len(lincs_set.intersection(landmark_set))


,gene_id,ensembl_id,gene_title,gene_type,src,feature_space
gene_symbol,,,,,,
A1CF,29974,ENSG00000148584,APOBEC1 complementation factor,protein-coding,NCBI,inferred
A2M,2,ENSG00000175899,alpha-2-macroglobulin,protein-coding,NCBI,best inferred
A4GALT,53947,ENSG00000128274,"alpha 1,4-galactosyltransferase (P blood group)",protein-coding,NCBI,best inferred
A4GNT,51146,ENSG00000118017,"alpha-1,4-N-acetylglucosaminyltransferase",protein-coding,NCBI,inferred
AAAS,8086,ENSG00000094914,aladin WD repeat nucleoporin,protein-coding,NCBI,best inferred
...,...,...,...,...,...,...
ZXDB,158586,ENSG00000198455,zinc finger X-linked duplicated B,protein-coding,NCBI,inferred
ZXDC,79364,ENSG00000070476,ZXD family zinc finger C,protein-coding,NCBI,best inferred
ZYX,7791,ENSG00000159840,zyxin,protein-coding,NCBI,best inferred


951

### Passing annotations

In [9]:
adata.obs['Sample'] = adata.obs['CellID'].str.split('_').str[0]
adata.obs.index = adata.obs['CellID']

In [10]:
adata.obs['Type'] = 'N/A'
for i in ['GSM5288668', 'GSM5288669']:
    adata.obs.loc[adata.obs['Sample'].str.contains(i, na=False), 'Type'] = 'NMIBC'
for i in ['GSM5288670', 'GSM5288671']:
    adata.obs.loc[adata.obs['Sample'].str.contains(i, na=False), 'Type'] = 'MIBC'    
for i in ['GSM5288672', 'GSM5288674']:
    adata.obs.loc[adata.obs['Sample'].str.contains(i, na=False), 'Type'] = 'Healthy'    

In [11]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata,base=2)
print(f'Max val after log1p, {np.max(adata.to_df())}')


Max val after log1p, 9.472813606262207


### Subset using Lincs landmark genes 

In [13]:
adata_lincs = adata[:, adata.var['Lincs'].notnull()]

In [24]:
adata_lincs

View of AnnData object with n_obs × n_vars = 15954 × 952
    obs: 'CellID', 'Sample', 'Type'
    var: 'Gene', 'Lincs'
    uns: 'log1p'

In [26]:
adata_lincs.obs['']

,CellID,Sample,Type
CellID,,,
GSM5288668_AAACCCACACTACTTT-1,GSM5288668_AAACCCACACTACTTT-1,GSM5288668,NMIBC
GSM5288668_AAACCCATCGGTCTGG-1,GSM5288668_AAACCCATCGGTCTGG-1,GSM5288668,NMIBC
GSM5288668_AAACGAACACACACTA-1,GSM5288668_AAACGAACACACACTA-1,GSM5288668,NMIBC
GSM5288668_AAACGAAGTGTTACAC-1,GSM5288668_AAACGAAGTGTTACAC-1,GSM5288668,NMIBC
GSM5288668_AAACGCTCACAATGCT-1,GSM5288668_AAACGCTCACAATGCT-1,GSM5288668,NMIBC
...,...,...,...
GSM5288674_TTTGATCGTACGAGCA-1,GSM5288674_TTTGATCGTACGAGCA-1,GSM5288674,Healthy
GSM5288674_TTTGATCGTTGCTGAT-1,GSM5288674_TTTGATCGTTGCTGAT-1,GSM5288674,Healthy
GSM5288674_TTTGGAGCACCTTCCA-1,GSM5288674_TTTGGAGCACCTTCCA-1,GSM5288674,Healthy


In [23]:
adata_lincs.obs['Type'].value_counts()

Type
MIBC       6783
NMIBC      3919
Healthy    3314
N/A        1938
Name: count, dtype: int64

In [15]:
#MIBC
MIBC= adata_lincs[adata_lincs.obs['Type'] == 'MIBC']
display(MIBC.obs)

,CellID,Sample,Type
CellID,,,
GSM5288670_AAACCCAAGCTAGATA-1,GSM5288670_AAACCCAAGCTAGATA-1,GSM5288670,MIBC
GSM5288670_AAACCCAAGTATAACG-1,GSM5288670_AAACCCAAGTATAACG-1,GSM5288670,MIBC
GSM5288670_AAACCCAAGTTGAAAC-1,GSM5288670_AAACCCAAGTTGAAAC-1,GSM5288670,MIBC
GSM5288670_AAACGAAAGGTAGCCA-1,GSM5288670_AAACGAAAGGTAGCCA-1,GSM5288670,MIBC
GSM5288670_AAACGAACACACACGC-1,GSM5288670_AAACGAACACACACGC-1,GSM5288670,MIBC
...,...,...,...
GSM5288671_TTTGTTGCACGACGAA-1,GSM5288671_TTTGTTGCACGACGAA-1,GSM5288671,MIBC
GSM5288671_TTTGTTGGTGAGACCA-1,GSM5288671_TTTGTTGGTGAGACCA-1,GSM5288671,MIBC
GSM5288671_TTTGTTGGTGGTCTAT-1,GSM5288671_TTTGTTGGTGGTCTAT-1,GSM5288671,MIBC


In [17]:
#NMIBC
NMIBC= adata_lincs[adata_lincs.obs['Type'] == 'NMIBC']
display(NMIBC.obs)

,CellID,Sample,Type
CellID,,,
GSM5288668_AAACCCACACTACTTT-1,GSM5288668_AAACCCACACTACTTT-1,GSM5288668,NMIBC
GSM5288668_AAACCCATCGGTCTGG-1,GSM5288668_AAACCCATCGGTCTGG-1,GSM5288668,NMIBC
GSM5288668_AAACGAACACACACTA-1,GSM5288668_AAACGAACACACACTA-1,GSM5288668,NMIBC
GSM5288668_AAACGAAGTGTTACAC-1,GSM5288668_AAACGAAGTGTTACAC-1,GSM5288668,NMIBC
GSM5288668_AAACGCTCACAATGCT-1,GSM5288668_AAACGCTCACAATGCT-1,GSM5288668,NMIBC
...,...,...,...
GSM5288669_TTTGGTTTCAGTGTGT-1,GSM5288669_TTTGGTTTCAGTGTGT-1,GSM5288669,NMIBC
GSM5288669_TTTGTTGAGCAGCCCT-1,GSM5288669_TTTGTTGAGCAGCCCT-1,GSM5288669,NMIBC
GSM5288669_TTTGTTGGTGAGAACC-1,GSM5288669_TTTGTTGGTGAGAACC-1,GSM5288669,NMIBC


In [18]:
#healthy
healthy= adata_lincs[adata_lincs.obs['Type'] == 'Healthy']
display(healthy.obs)

,CellID,Sample,Type
CellID,,,
GSM5288672_AAACCCACAGCTATTG-1,GSM5288672_AAACCCACAGCTATTG-1,GSM5288672,Healthy
GSM5288672_AAACCCATCTTCCGTG-1,GSM5288672_AAACCCATCTTCCGTG-1,GSM5288672,Healthy
GSM5288672_AAACGAAAGCGTCTCG-1,GSM5288672_AAACGAAAGCGTCTCG-1,GSM5288672,Healthy
GSM5288672_AAAGAACCACTCTAGA-1,GSM5288672_AAAGAACCACTCTAGA-1,GSM5288672,Healthy
GSM5288672_AAAGGGCCACCAATTG-1,GSM5288672_AAAGGGCCACCAATTG-1,GSM5288672,Healthy
...,...,...,...
GSM5288674_TTTGATCGTACGAGCA-1,GSM5288674_TTTGATCGTACGAGCA-1,GSM5288674,Healthy
GSM5288674_TTTGATCGTTGCTGAT-1,GSM5288674_TTTGATCGTTGCTGAT-1,GSM5288674,Healthy
GSM5288674_TTTGGAGCACCTTCCA-1,GSM5288674_TTTGGAGCACCTTCCA-1,GSM5288674,Healthy


In [ ]:
#For raw data retrival 
def subset(data,name):
    mtx =f"{data}_filtered_matrix.mtx.gz"
    cells=pd.read_csv(f"{data}_filtered_barcodes.tsv.gz",header=None)
    features=pd.read_csv(f'{data}_filtered_features.tsv.gz',header=None,sep='\t')
    adata = sc.read_mtx(mtx)
    
    adata= adata.T
    print(adata)
    adata.obs['CellID']= cells[0].tolist()
    adata.var['Gene']= features[1].tolist()
    adata.var.index= adata.var['Gene']
    
    adata.var_names_make_unique()
    barcodes_subset = barcodes_uro.loc[barcodes_uro['Sample'] == name, 'Barcode']
    adata_subset = adata[adata.obs['CellID'].isin(barcodes_subset)]
    print(adata_subset)
    adata_subset = adata_subset[:, adata_subset.var_names.isin(gene_list)]
    sc.pp.normalize_total(adata_subset, target_sum=1e4)
    sc.pp.log1p(adata_subset,base=2)
    adata_subset_df=adata_subset.to_df()
   # adata_subset_df.to_pickle(f"{name}_sub.pkl")
